<a href="https://colab.research.google.com/github/peterjsadowski/Tutorial-Microbiome/blob/main/part2_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning for Microbiome Data Analysis

This notebook presents a tutorial on using machine learning with sklearn for analyzing microbiome amplicon data.

Author: Peter Sadowski

Date: Feb 19 2023


# Loading Data

For this example we use samples taken by UH Professor Anthony Amend's BOT662 class. Instructions:
1. Download file from [this link](https://drive.google.com/drive/u/0/folders/1ueHCzkohAkXOW-ZcbqDgeLr8PnQ5Rxyd) (Google login required).
1. Upload the zip file 'Class_Data_for_Phyloseq.zip' to the runtime.
1. Unzip it below.

In [ ]:
!unzip Class_Data_for_Phyloseq.zip  # Assumes file is saved locally.

In [2]:
import numpy as np
import pandas as pd
import sklearn
import matplotlib.pylab as plt

# Load data into pandas dataframe.
df = pd.read_csv('OTUs.100.rep.count_table.csv')
df = df.rename(columns={'Unnamed: 0': 'OTU'})
df = df.set_index('OTU')

meta_data = pd.read_csv('brom_meta.csv')
meta_data = meta_data.set_index('sample_name')

# Supervised machine learning

In supervised learning, the goal is to predict some target variable _y_ from some input variable _x_. In a probabilistic machine learning model, the predictions will be in the form of a conditional probability distribution p(y|x), where this might be a discrete probability mass function over a finite set of class labels (classification) or a probability density function over real values (regression). Classification is more concrete, so we focus on it here.


# Logistic Regression Classifier
Below we use a simple __logistic regression__ classifier to predict the sample_type from amplicon counts. 



In [35]:
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import LabelEncoder

def assign_colors(df, meta_data, colorby='sample_type'):
  """ Assign a integer label and color to each unique value in the colorby col.
  Returns:
    labels = List of labels. Has length equal to len(df.columns).
    labels_int = List of integer labels.
    colors = List of colors. Has length equal to len(df.columns).
    label_colors = Dict mapping unique keys to colors.
  """
  labels = [meta_data.loc[[sample_name]][colorby][0]
          for sample_name in df.columns]
  encoder = LabelEncoder()
  labels_int = encoder.fit_transform(labels)
  cmap = plt.get_cmap('Accent')
  num_labels = len(np.unique(labels))
  label_colors = {label: cmap(i/num_labels) 
                  for i, label in enumerate(np.unique(labels))}
  colors = [label_colors[label] for label in labels]
  return labels, labels_int, colors, label_colors

labels, labels_int, colors, label_colors = assign_colors(
    df, meta_data, colorby='sample_type')

# Classification task is to predict sample_type from 
X = (np.log(1+df).T).to_numpy()
y = labels_int 

clf = LogisticRegression(random_state=0, penalty='l2', max_iter=1000).fit(X, y)
# This function predicts the most likely class label.
labels_predicted = clf.predict(X[:, :])
# This function gives probabilities for each class.
probs_predicted = clf.predict_proba(X[:, :])

print(f'Accuracy: {clf.score(X, y) * 100}%')

Accuracy: 100.0%


## Cross-Validation
This model gets 100% accuracy on our little dataset of 22 examples. Is that good? We _don't know_ without testing it on new data! Machine learning models, even simple ones like this linear model, are prone to __overfit__ to the data they are trained on. A powerful model can _always_ get 100% on the training data, but that doesn't mean the model __generalizes__ to new examples (which is the only thing we ever care about). Thus, the standard way to evaluate ML models is through __cross-validation__ where the data is separated into a __training set__ that is used to fit the model and __test set__ that is only used for evaluating the model performance.

Scikit learn has some very helpful methods managing these data splits. For example, train_test_split method splits off 40% of the data to be in the test set.


In [46]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4)
clf = LogisticRegression(penalty='l2', max_iter=1000).fit(X_train, y_train)
print(f'Accuracy: {clf.score(X_test, y_test) * 100}%')

Accuracy: 33.33%


This is far less than the 100% accuracy from before. This shouldn't be surprising, since we are only training on ~13 examples! Isn't there a way to use more of our tiny dataset for training? Unfortunatley, if we decrease the size of our test set, we get a very noisy estimate of the performance. Indeed, if the test set contains only one example, then the accuracy estimate will be either 0% or 100%.

There is a nice trick that gives us the best of both worlds, at the cost of more computation. In __K-fold cross-validation__, we split the data into k subsets, and train k models, using a different subset for testing each iteration. In the extreme case where k is the number of samples in the dataset, this is called __leave-one-out cross validation__.  

In [ ]:
from sklearn import metrics
from sklearn.model_selection import cross_val_score

clf = LogisticRegression(random_state=0, penalty='l2', max_iter=1000)
%timeit scores = cross_val_score(clf, X, y, cv=5)
print(f'CV accuracies: {scores}')
print(f'Mean accuracy: {scores.mean():.2f}')  # prints only two decimals

By default, Scikit Learn attempts to do __stratified cross-validation__, in which the examples in each class are evenly distributed between the k-folds. 

In order to do leave-one out cross-validation, we can pass the LeaveOneOut iterator object to cross_val_score. Notice that this takes a minute to run, because it is training 22 different models. 

In [40]:
from sklearn.model_selection import LeaveOneOut
clf = LogisticRegression(random_state=0, penalty='l2', max_iter=1000)
%timeit scores = cross_val_score(clf, X, y, cv=LeaveOneOut())
print(f'CV accuracies: {scores}')
print(f'Mean accuracy: {scores.mean():.2f}')

25 s ± 1.96 s per loop (mean ± std. dev. of 7 runs, 1 loop each)
CV accuracies: [0.6  0.8  1.   1.   0.75]
Mean accuracy: 0.8300000000000001


# Decision Trees

One of the most intuitive and interpretable machine learning algorithms is the decision tree. Training a decision tree consists of iteratively splitting the data space and assigning labels to each section.

![decision boundary](https://scikit-learn.org/stable/_images/sphx_glr_plot_iris_dtc_001.png)



In [ ]:
from sklearn.tree import DecisionTreeClassifier

# Suppress those warnings about having a class with only one example.
import warnings
warnings.filterwarnings("ignore", category=UserWarning)

clf = DecisionTreeClassifier(max_depth=3)
%timeit scores = cross_val_score(clf, X, y, cv=5)
print(f'CV accuracies: {scores}')
print(f'Mean accuracy: {scores.mean():.2f}')